In [1]:
import cv2 as cv
import numpy as np
import string
import os
from copy import deepcopy

In [2]:

NO_CELLS = 15
CELL_WIDTH = 150

#create 2 new directories, 1 will hold letters for comparission, 2 will hold our predictions 

letters_path = '../letters'
if not os.path.exists(letters_path):
    os.makedirs(letters_path)
    
results_path = '../predictions/331_Hadirca_Dionisie'
if not os.path.exists(results_path):
    os.makedirs(results_path)

In [3]:

# used functions

def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def get_text_data(filepath):
    res = []
    with open(filepath) as f:
        row = f.readline()
        while row!='': 
            res.append([i for i in row.split()] )
            row = f.readline()
    return res



def extract_grid(image):
    work_image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(work_image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur,(0,0),5)
    image_sharpened = cv.addWeighted(image_m_blur,1.5,image_g_blur,-0.8,0)
    
    _,thresh = cv.threshold(image_sharpened,130,255,cv.THRESH_BINARY)
    
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.dilate(thresh, kernel)
    
    edges = cv.Canny(thresh,1,120)
    edges = cv.dilate(edges,kernel)
    
    contours, _ =cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    image_copy = deepcopy(image)
    cv.drawContours(image_copy, contours, -1, (0,255,0), 3)
    

    max_area = 0
    
    top_left = [0,0]
    top_right = [0,0]
    bottom_right = [0,0]
    bottom_left = [0,0]
    for i in range(len(contours)):
        if len(contours[i]) > 3:
            possible_top_left = None
            possible_bottom_right = None
            
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point
                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
                    possible_bottom_right = point
            
            diff = np.diff(contours[i].squeeze(), axis = 1)
            
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            
            calculated_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
            
            if calculated_area > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                top_right = possible_top_right
                bottom_right = possible_bottom_right
                bottom_left = possible_bottom_left
                
    image_copy = deepcopy(image)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    width = NO_CELLS * CELL_WIDTH
    height = NO_CELLS * CELL_WIDTH
    
    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")
    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)
    result = cv.warpPerspective(image, M, (width, height))
    
    return result



def classify_letter(patch):
    
    def transform_image(img):
        img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        image_m_blur = cv.medianBlur(img,3)
        image_g_blur = cv.GaussianBlur(image_m_blur,(0,0),3)
        img_sharpened = cv.addWeighted(image_m_blur,2,image_g_blur,-1,0)
        
        return img_sharpened
    

    maxi = -np.inf
    rez = ''
    
    
    patch_sharpened = transform_image(patch)
    
    for filename in os.listdir('../letters'):
        if filename[-3:] == 'jpg':
            lt = cv.imread('../letters/'+filename)
            lt_sharpened = transform_image(lt)
            
            
            corr = cv.matchTemplate(patch_sharpened,lt_sharpened[30:120,30:120],cv.TM_CCOEFF_NORMED)
            corr = np.max(corr)
            if corr > maxi:
                maxi = corr
                rez = filename[:-4]
    
    if rez == 'joker':
        return '?'
    return rez
            
def get_horizontal_left(board,i,j):
    left_part = []
    while j >= 0 and board[i][j] not in FREE_POSITIONS:
        left_part.insert(0,[board[i][j],(i,j)])
        j-=1
        
    return left_part

def get_horizontal_right(board,i,j):
    if j == 15:
        return []
    
    j+=1
    right_part = []
    while j < 15 and board[i][j] not in FREE_POSITIONS:
        right_part.append([board[i][j],(i,j)])
        j+=1
        
    return right_part

def get_vertical_up(board,i,j):
    up_part = []
    while i >= 0 and board[i][j] not in FREE_POSITIONS:
        up_part.insert(0,[board[i][j],(i,j)])
        i-=1
    return up_part

def get_vertical_down(board,i,j):
    if i == 15:
        return []
    
    i+=1
    down_part = []
    while i < 15 and board[i][j] not in FREE_POSITIONS:
        down_part.append([board[i][j],(i,j)])
        i+=1
        
    return down_part

def calculate_word_value(current_word):
    word_score = 0
    multiplicator = 1
    
    for comb in current_word:
        letter_score = score_dict[comb[0]]
        pos_bonus = calc_board[comb[1][0]][comb[1][1]]

        if pos_bonus == RED:
            multiplicator*=3
        elif pos_bonus == PINK:
            multiplicator*=2
        elif pos_bonus == BLUE:
            letter_score*=3
        elif pos_bonus == SKY:
            letter_score*=2

        word_score += letter_score

    word_score *= multiplicator
  
    return word_score

In [5]:
# this directory holds images with the plays
train_directory = '../images'

images = []
info = []
for filename in os.listdir(train_directory):
    if filename[-3:] == 'jpg':
        img = cv.imread(train_directory+'/'+filename)
        images.append(extract_grid(img))
    else:
        info.append(get_text_data(train_directory+'/'+filename))

In [6]:
train_data = []
train_info = []

for i in range(5):
    train_data.append(images[i*20:(i*20)+20])
    
    if(train_info != []):
        train_info.append(info[i*20:(i*20)+20])

In [7]:
# from this directory we extract our letters 

auxiliary = '../imagini_auxiliare'
letters_img = cv.imread(auxiliary+'/litere_1.jpg')
letters = extract_grid(letters_img)[5*CELL_WIDTH:10*CELL_WIDTH,5*CELL_WIDTH:10*CELL_WIDTH]


game_alphabet = string.ascii_uppercase
for i in 'KWQY':
    game_alphabet = game_alphabet.replace(i,'')

os.chdir('../letters')

for i in range(5):
    for j in range(5):
        xCoord = (i*CELL_WIDTH)
        yCoord = (j*CELL_WIDTH)
        patch = deepcopy(letters[xCoord:xCoord+CELL_WIDTH,yCoord:yCoord+CELL_WIDTH])
        idx = (i*5)+j
        if idx < 22:
            cv.imwrite(game_alphabet[idx]+'.jpg', patch)
        elif idx == 22:
            cv.imwrite('joker.jpg',patch)
        else:
            break

In [8]:

RED = 'r'
PINK = 'p'
BLUE = 'b'
SKY = 's'
OCCUPIED = -1
FREE_POSITIONS = [RED,PINK,BLUE,SKY,0]

scrabble_board =[
    [RED, 0, 0, SKY, 0, 0, 0, RED, 0, 0, 0, SKY, 0, 0, RED],
    [0, PINK, 0, 0, 0, BLUE, 0, 0, 0, BLUE, 0, 0, 0, PINK, 0],
    [0, 0, PINK, 0, 0, 0, SKY, 0, SKY, 0, 0, 0, PINK, 0, 0],
    [SKY, 0, 0, PINK, 0, 0, 0, SKY, 0, 0, 0, PINK, 0, 0, SKY],
    [0, 0, 0, 0, PINK, 0, 0, 0, 0, 0, PINK, 0, 0, 0, 0],
    [0, BLUE, 0, 0, 0, BLUE, 0, 0, 0, BLUE, 0, 0, 0, BLUE, 0],
    [0, 0, SKY, 0, 0, 0, SKY, 0, SKY, 0, 0, 0, SKY, 0, 0],
    [RED, 0, 0, SKY, 0, 0, 0, PINK, 0, 0, 0, SKY, 0, 0, RED],
    [0, 0, SKY, 0, 0, 0, SKY, 0, SKY, 0, 0, 0, SKY, 0, 0],
    [0, BLUE, 0, 0, 0, BLUE, 0, 0, 0, BLUE, 0, 0, 0, BLUE, 0],
    [0, 0, 0, 0, PINK, 0, 0, 0, 0, 0, PINK, 0, 0, 0, 0],
    [SKY, 0, 0, PINK, 0, 0, 0, SKY, 0, 0, 0, PINK, 0, 0, SKY],
    [0, 0, PINK, 0, 0, 0, SKY, 0, SKY, 0, 0, 0, PINK, 0, 0],
    [0, PINK, 0, 0, 0, BLUE, 0, 0, 0, BLUE, 0, 0, 0, PINK, 0],
    [RED, 0, 0, SKY, 0, 0, 0, RED, 0, 0, 0, SKY, 0, 0, RED],
]

In [9]:
points = ['1ACEILNORSTU','2DP','4M','8FV','9BG','XHJXZ']

score_dict = {}
for value in points:
    if value[0] == 'X':
        for let in value[1:]:
            score_dict[let] = 10
    else:
        for let in value[1:]:
            score_dict[let] = int(value[0])
            
score_dict['?'] = 0

In [10]:
played_positions = []
idx = 0

for data_set in train_data:
    board = deepcopy(scrabble_board)
    calc_board = deepcopy(scrabble_board)
    
    for im in data_set:
        l = []
        for i in range(NO_CELLS):
            for j in range(NO_CELLS):
                patch = deepcopy(im[
                    (i*CELL_WIDTH):(i*CELL_WIDTH)+CELL_WIDTH,
                    (j*CELL_WIDTH):(j*CELL_WIDTH)+CELL_WIDTH
                ])
                
                modif_patch = cv.cvtColor(patch[20:130,20:130],cv.COLOR_BGR2GRAY)
                modif_patch = cv.GaussianBlur(modif_patch,(0,0),5)
                modif_patch = cv.addWeighted(modif_patch,1.5,modif_patch,-0.8,0)
                _,modif_patch = cv.threshold(modif_patch,130,255,cv.THRESH_BINARY)
                
                patch_mean = np.mean(modif_patch)
                
                # here we get positions and letter played in one round
                if patch_mean > 0 and board[i][j] in FREE_POSITIONS: 
                    
                    letter = classify_letter(patch)
                    l.append([str(i+1)+string.ascii_uppercase[j],letter])
                    board[i][j] = letter
        
        played_positions.append(l)
        
        # from here we calculate the score of the created words
        
        calculated_words = []
        
        word_score_horizontal = 0
        word_score_vertical = 0
        word_score = 0
        
        
        # for each letter we would get its horizontal and vertical word_score combinations and will sum it into a total one
        
        for (pos,letter) in l:
                    
            i = int(pos[0:-1]) - 1
            j = string.ascii_uppercase.index(pos[-1])
            
            current_horizontal_word = get_horizontal_left(board,i,j) + get_horizontal_right(board,i,j)  
            
            if current_horizontal_word not in calculated_words and len(current_horizontal_word) > 1:
                
                word_score_horizontal += calculate_word_value(current_horizontal_word)
                calculated_words.append(current_horizontal_word)
            
            

            current_vertical_word = get_vertical_up(board,i,j) + get_vertical_down(board,i,j)
            
            if current_vertical_word not in calculated_words and len(current_vertical_word) > 1:
                
                word_score_vertical += calculate_word_value(current_vertical_word)
                calculated_words.append(current_vertical_word)
                
            
            word_score = word_score_horizontal + word_score_vertical
            

        played_positions[idx].append(word_score)
        
        # we then place each word on the board and save boards state as occupied at the respective positions
        for word in calculated_words:
            for comb in word:
                calc_board[comb[1][0]][comb[1][1]] = OCCUPIED
    
        idx+=1

In [10]:
# in this cell we test our code if we have train_labels

# check = 0

# for row in range(100):
#     if(len(info[row]) != len(played_positions[row])):
#         continue
#     tru = 1
#     for i in range(len(info[row][:-1])):
#         tru &= info[row][i][0] == played_positions[row][i][0]
#     check+=tru
# print('correct positions: ',check)


# check = 0

# for row in range(100):
#     if(len(info[row]) != len(played_positions[row])):
#         continue
#     tru = 1
#     for i in range(len(info[row][:-1])):
#         tru &= info[row][i][1] == played_positions[row][i][1]
#     check+=tru
# print('correct letters:',check)


# check = 0

# for row in range(100):
#     if(len(info[row]) != len(played_positions[row])):
#         continue
    
#     check += (int(info[row][-1][0]) == int(played_positions[row][-1]))
    
#     if (int(info[row][-1][0]) != int(played_positions[row][-1])):
#         print(info[row],'    ',played_positions[row], row)
# print('correct score:',check)

In [11]:
for i in range(len(played_positions)):
    
    fn = (i%20) + 1
    if fn < 10:
        fn = '0' + str(fn)
    else:
        fn = str(fn)
    
    with open(f'{results_path}/{(i//20) + 1}_'+fn +'.txt', 'w') as g:
        for conf in played_positions[i][:-1]:
            g.write(conf[0] + ' ' + conf[1] + '\n')
        g.write(str(played_positions[i][-1]))
            
        

[['8H', 'P'], ['8I', 'U'], ['8J', 'N'], ['8K', 'C'], ['8L', 'T'], 14]
[['9K', 'E'], ['10K', 'R'], ['11K', 'T'], 8]
[['4L', 'A'], ['5L', 'L'], ['6L', 'E'], ['7L', 'R'], 10]
[['11L', 'U'], ['11M', 'R'], ['11N', 'T'], ['11O', 'I'], 5]
[['4K', 'H'], ['4M', 'O'], ['4N', 'S'], 13]
[['12M', 'A'], ['13M', 'M'], ['14M', 'A'], 14]
[['6I', 'V'], ['6J', 'E'], ['6K', 'N'], ['6M', 'A'], 14]
[['14H', 'N'], ['14I', 'E'], ['14J', 'G'], ['14K', 'U'], ['14L', 'R'], 32]
[['6G', 'R'], ['6H', 'E'], 14]
[['9H', 'A'], ['10H', 'L'], ['11H', 'I'], 5]
[['10O', 'Z'], ['12O', 'S'], 13]
[['9L', 'A'], 8]
[['11C', 'D'], ['11D', 'O'], ['11E', 'C'], ['11F', 'I'], ['11G', 'L'], 14]
[['15G', 'G'], ['15H', 'E'], ['15I', 'N'], 41]
[['13K', 'B'], 10]
[['4G', 'S'], ['5G', 'I'], 3]
[['4C', 'S'], ['4D', 'I'], ['4E', 'N'], ['4F', 'U'], 10]
[['5C', 'E'], ['6C', 'T'], 3]
[['15K', 'C'], 11]
[['10D', 'P'], ['12D', 'D'], 10]
[['8D', 'S'], ['8E', 'U'], ['8F', '?'], ['8G', 'I'], ['8H', 'T'], ['8I', 'E'], 12]
[['7G', 'F'], ['9G', 'N'],